In [1]:
import sqlite3
from datetime import datetime
from typing import List

In [13]:
class Account:
    """
    Класс банковского аккаунта, предназнеченный для заведения нового участника, внесения и снятия денег с счета.
    Предусмотрено ведение истории операций с возможностью просмотра последних 100 операций.
    """

    def __init__(self, name: str, start_balance: int = 0) -> None:
        self.name = name
        self.start_balance = start_balance

        self.__create_table()

    def __create_table(self):
        with sqlite3.connect('bank.db') as conn:
            cursor = conn.cursor()
            cursor.execute("""CREATE TABLE IF NOT EXISTS accounts_history (
                                                    ID INTEGER PRIMARY KEY AUTOINCREMENT,
                                                    name TEXT NOT NULL,
                                                    operation_name TEXT NOT NULL,
                                                    balance INT NOT NULL,
                                                    date TEXT DEFAULT CURRENT_TIMESTAMP
                                                    )
                            """)
            conn.commit()

    def deposit_money(self, add_money: int):
        if type(add_money) == int and add_money > 0:
            self.start_balance += add_money
            with sqlite3.connect("bank.db") as conn:
                cursor = conn.cursor()
                cursor.execute("""INSERT INTO accounts_history (name, operation_name, balance) VALUES (?, ?, ?)""",
                               (self.name, 'deposit_money', self.start_balance))
                conn.commit()
            return print(f"Внесено {add_money} ед.\nТекущий баланс {self.start_balance}")

        return print("Введены некорректные данные")

    def get_money(self, push_money: int):
        if self.start_balance <= push_money:
            return print(f"У вас недостаточно средств.\nТекущий баланс {self.start_balance} ед")
        self.start_balance -= push_money

        with sqlite3.connect("bank.db") as conn:
            cursor = conn.cursor()
            cursor.execute("""INSERT INTO accounts_history (name, operation_name, balance) VALUES (?, ?, ?)""",
                           (self.name, 'get_money', self.start_balance))
            conn.commit()

        return print(f"Выдано {push_money} ед.\nТекущий баланс {self.start_balance}")

    def get_history(self) -> List:
        with sqlite3.connect('bank.db') as conn:
            cursor = conn.cursor()
            cursor.execute(f"""
                           SELECT  ID, name, operation_name, balance, strftime('%d.%m.%Y %H:%M', date)
                           FROM accounts_history 
                           where name = '{self.name}' order by date desc
                           LIMIT 100
                           """)
            return cursor.fetchall()

In [15]:
ivan = Account('Ivan', 100)
any = Account("Any", 1000)

In [17]:
any.deposit_money(100)

Внесено 100 ед.
Текущий баланс 1100


In [19]:
ivan.get_money(10)

Выдано 10 ед.
Текущий баланс 90


In [21]:
ivan.get_money(1000)

У вас недостаточно средств.
Текущий баланс 90 ед


In [23]:
ivan.deposit_money(1000)

Внесено 1000 ед.
Текущий баланс 1090


In [27]:
print(ivan.get_history())

[(5, 'Ivan', 'deposit_money', 1090, '02.10.2024 21:26'), (4, 'Ivan', 'get_money', 90, '02.10.2024 21:25')]


In [29]:
print(any.get_history())

[(3, 'Any', 'deposit_money', 1100, '02.10.2024 21:25'), (2, 'Any', 'deposit_money', 1200, '02.10.2024 21:25'), (1, 'Any', 'deposit_money', 1100, '02.10.2024 21:24')]
